# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unraveling the World of Deep Learning with DeepLearning.AI

Dive into the realm of artificial intelligence with DeepLearning.AI, a platform revolutionizing education in deep learning. Founded by Andrew Ng, a renowned figure in AI, DeepLearning.AI offers top-notch courses to upskill both beginners and experts in the field. From neural networks to machine learning, this platform covers it all. With interactive assignments and practical insights, learners gain hands-on experience essential for success in the AI industry. Whether you're a tech enthusiast or a professional looking to advance your skills, DeepLearning.AI equips you with the knowledge to thrive in the exciting world of deep learning.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unraveling the World of Deep Learning with DeepLearning.AI

Dive into the realm of artificial intelligence with DeepLearning.AI, a platform revolutionizing education in deep learning. Founded by Andrew Ng, a renowned figure in AI, DeepLearning.AI offers top-notch courses to upskill both beginners and experts in the field. From neural networks to machine learning, this platform covers it all. With interactive assignments and practical insights, learners gain hands-on experience essential for success in the AI industry. Whether you're a tech enthusiast or a professional looking to advance your skills, DeepLearning.AI equips you with the knowledge to thrive in the exciting world of deep learning.

-----------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unraveling the World of Deep Learning with DeepLearning.AI

Dive into the realm of artificial intelligence with DeepLearning.AI, a platform revolutionizing education in deep learning. Founded by Andrew Ng, a renowned figure in AI, DeepLearning.AI offers top-notch courses to upskill both beginners and experts in the field. From neural networks to machine learning, this platform covers it all. With interactive assignments and practical insights, learners gain hands-on experience essential for success in the AI industry. Whether you're a tech enthusiast or a professional looking to advance your skills, DeepLearning.AI equips you with the knowledge to thrive in the exciting world of deep learning.

-----------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated Feedback:
From the SEO Specialist: Focus on incorporating relevant keywords, optimizing meta titles and descriptions, and implementing structured data markup to improve search visibility.
From the Legal Reviewer: Ensure compliance with intellectual property laws, include a disclaimer about educational purposes, and review claims made to avoid misleading users.
From the Ethics Reviewer: Disclose potential conflicts of interest, verify the accuracy of claims made, and prioritize inclusivity in course accessibility.

Final Suggestions:
1. Implement the SEO Specialist's recommendations by adding relevant keywords, optimizing meta tags, and utilizing structured data markup for better search visibility.
2. Address the Legal Reviewer's concerns by verifying intellectual property compliance, including a disclaimer, and ensuring claims about offerings are accurate.
3. Follow 

## Get the summary

In [15]:
print(res.summary)

Title: Navigating Deep Learning Terrain with DeepLearning.AI

Embark on a transformative journey through DeepLearning.AI, a pioneering hub reshaping deep learning education. Spearheaded by AI luminary Andrew Ng, this platform offers an array of courses catering to all proficiency levels. From neural networks to advanced machine learning, DeepLearning.AI provides essential hands-on experience through interactive tasks and real-world insights. Whether you're stepping into tech or a seasoned pro honing your skills, DeepLearning.AI equips you to excel in the dynamic realm of AI. Stay ahead in artificial intelligence with the knowledge and expertise gained from DeepLearning.AI.
